# Score cell cycle

## Import statements

In [1]:
import os,sys
import datetime

In [2]:
import scanpy as sc
sc.logging.print_versions()
sc.logging.print_memory_usage()
sc.settings.verbosity = 2

scanpy==1.4.4.post1 anndata==0.6.22.post1 umap==0.3.7 numpy==1.15.4 scipy==1.3.1 pandas==0.23.4 scikit-learn==0.20.1 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1
Memory usage: current 0.20 GB, difference +0.20 GB


In [3]:
## This cell is run once to download my custom functions and import statements from github
#
#!git clone --depth=1 https://github.com/rapolaszilionis/utility_functions
#    
## github doesn't seem to have an option to download a specific version of the repo from the history.
## So I download my utility functions and save the download time by appending it to the directory name.
## These utility functions to be shared together with the notebook.
#
#toappend = datetime.datetime.now().strftime('%y%m%d_%Hh%M')
#newname = "utility_functions_%s"%toappend
#print(newname)
#
#
## rename the py file with utility functions
#os.rename("utility_functions",newname)

In [4]:
# add the utility function folder to PATH
sys.path.append(os.path.abspath("utility_functions_210525_14h42/"))

from rz_import_statements import *
import rz_functions as rz
import rz_fig_params # this adjust mpl.rcParams, almost nothing to import, import after scanpy to overwrite rc.Params
import rz_utility_spring as srz

python version: 3.6.7


## Load data

In [5]:
adata = sc.read_h5ad('backups/mito_total_counts_filt_raw_32415x36601_210622_20h17.h5ad')

In [6]:
# load obs with classifier results
adata.obs = rz.load_df('backups/obs_info_32415x9_210622_20h17.npz')

## Download cell cycle gene list

In [7]:
#!mkdir -p Seurat_cell_cycle
# download cell cycles genes from here:
# https://www.dropbox.com/s/3dby3bjsaf5arrw/cell_cycle_vignette_files.zip?dl=1
# place into Seural_cell_cycle, unzip

ccpath = 'Seurat_cell_cycle/cell_cycle_vignette_files/regev_lab_cell_cycle_genes.txt'
cell_cycle_genes = np.loadtxt(ccpath,dtype='str')

print(len(cell_cycle_genes),len(set(cell_cycle_genes)))

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

97 97


## Calculate cell cycle scores

In [8]:
# select cells to consider
cell_mask = np.repeat(True,adata.shape[0]) # all cells in this case
print(cell_mask.sum(),len(cell_mask))

32415 32415


In [9]:
# make a copy of adata only focusing on cells of interest
cdata = adata[cell_mask,:].copy()

# filter out genes with zero counts
min_non_zero = cdata.X.data.min()
sc.pp.filter_genes(cdata,min_counts=min_non_zero)

# scale to 1e4 total counts
sc.pp.normalize_per_cell(cdata, counts_per_cell_after=1e4)

# log transform
sc.pp.log1p(cdata)

# zscore
sc.pp.scale(cdata)

filtered out 11723 genes that are detectedin less than 1.0 counts
normalizing by total count per cell
    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


In [10]:
# make sure to only leave those cell cycle genes I have in the reduced dataset
print(len(s_genes),len(g2m_genes),len(cell_cycle_genes))
s_genes = [i for i in s_genes if i in cdata.var_names]
g2m_genes = [i for i in g2m_genes if i in cdata.var_names]
cell_cycle_genes = [i for i in cell_cycle_genes if i in cdata.var_names]
print(len(s_genes),len(g2m_genes),len(cell_cycle_genes))

43 54 97
42 52 94


In [11]:
help(sc.tl.score_genes)

Help on function score_genes in module scanpy.tools._score_genes:

score_genes(adata, gene_list, ctrl_size=50, gene_pool=None, n_bins=25, score_name='score', random_state=0, copy=False, use_raw=False)
    Score a set of genes [Satija15]_.
    
    The score is the average expression of a set of genes subtracted with the
    average expression of a reference set of genes. The reference set is
    randomly sampled from the `gene_pool` for each binned expression value.
    
    This reproduces the approach in Seurat [Satija15]_ and has been implemented
    for Scanpy by Davide Cittaro.
    
    Parameters
    ----------
    adata : :class:`~anndata.AnnData`
        The annotated data matrix.
    gene_list : iterable
        The list of gene names used for score calculation.
    ctrl_size : `int`, optional (default: 50)
        Number of reference genes to be sampled. If `len(gene_list)` is not too
        low, you can set `ctrl_size=len(gene_list)`.
    gene_pool : `list` or `None`, optio

In [12]:
help(sc.tl.score_genes_cell_cycle)

Help on function score_genes_cell_cycle in module scanpy.tools._score_genes:

score_genes_cell_cycle(adata, s_genes, g2m_genes, copy=False, **kwargs)
    Score cell cycle genes [Satija15]_.
    
    Given two lists of genes associated to S phase and G2M phase, calculates
    scores and assigns a cell cycle phase (G1, S or G2M). See
    :func:`~scanpy.api.score_genes` for more explanation.
    
    Parameters
    ----------
    adata : :class:`~anndata.AnnData`
        The annotated data matrix.
    s_genes : `list`
        List of genes associated with S phase.
    g2m_genes : `list`
        List of genes associated with G2M phase.
    copy : `bool`, optional (default: `False`)
        Copy `adata` or modify it inplace.
    **kwargs : optional keyword arguments
        Are passed to :func:`~scanpy.api.score_genes`. `ctrl_size` is not
        possible, as it's set as `min(len(s_genes), len(g2m_genes))`.
    
    Returns
    -------
    Depending on `copy`, returns or updates `adata` wit

In [13]:
# score cell cycle
sc.tl.score_genes_cell_cycle(cdata, s_genes=s_genes, g2m_genes=g2m_genes, random_state=1)

calculating cell cycle phase
computing score 'S_score'
    finished (0:00:27)
computing score 'G2M_score'
    finished (0:00:30)


In [14]:
newcols = [i for i in cdata.obs.columns if i not in adata.obs.columns]

In [15]:
newcolobs = cdata.obs[newcols]
newcolobs.head()

,n_counts,S_score,G2M_score,phase
82,2560.0,-0.078621,-0.034290,G1
121,1383.0,0.008152,0.062619,G2M
130,913.0,-0.044780,0.358308,G2M
132,3974.0,0.136416,-0.079504,S
174,1988.0,0.036697,-0.057833,S


## Rename the resulting scores  

Thanks to my colleague MM for pointing out the following. 


"Oké, I will keep the function explanation but will name the sc.tl.score_genes_cell_cycle function. [...]
I should note that I think the Scanpy interpretation of the papers on which this analysis is based is not completely right. “S genes” should be called “G1/S genes” and the G1 classification that they classify as both G2M and S gene signature scores are negative in their code is actually a G0 classification (see Fig. 2F from the paper that derived the gene sets that Tirosh et al., 2016 used): https://genome.cshlp.org/content/25/12/1860.full#F2. The finding from this paper is that G1/S and S phase signatures and G2 and G2/M signatures (mostly defined by Whitfield et al., 2002) are not separated well by clustering and that G1/S and G2/M signatures are expressed by proliferating cells. Therefore, G1/S and G2/M signatures are the resolution that we can get from single cell expression and should be used to classify proliferating cells as these cells must express either or both of these signatures (as was done in Fig. 2A Tirosh et al., 2016: https://science.sciencemag.org/content/352/6282/189). When cells do not express G1/S, or G2/M signatures they should be classified as non-proliferating cells that are G0 and not G1, which is a phase that is part of the cell cycle. "

In [16]:
prenamer = {"S":"G1S",
           "G1":"G0",
           "G2M":"G2M"}

colrenamer = {"S_score":"G1S_score"}

newcolobs.columns = [colrenamer[i] if i in colrenamer else i for i in newcolobs.columns]
newcolobs['phase'] = [prenamer[i] for i in newcolobs['phase']]

/Users/rapolaszilionis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
newcolobs.head()

,n_counts,G1S_score,G2M_score,phase
82,2560.0,-0.078621,-0.034290,G0
121,1383.0,0.008152,0.062619,G2M
130,913.0,-0.044780,0.358308,G2M
132,3974.0,0.136416,-0.079504,G1S
174,1988.0,0.036697,-0.057833,G1S


## Save update obs

In [18]:
for col in newcolobs.columns:
    adata.obs[col] = newcolobs[col].values

In [19]:
# no need to save the entire adata object, counts didn't change

fname = 'backups/obs_info_%dx%d_%s'%(adata.obs.shape[0],adata.obs.shape[1],rz.now())
print(fname)
rz.save_df(adata.obs,fname)

backups/obs_info_32415x13_210821_11h29


## Add the new colotracks to SPRING

In [58]:
path1 = "/Users/rapolaszilionis/Google Drive/DG/tmp_for_S3/20210616_RZ_macrophage_fastq_and_counts/quick_analysis_spring_plot/SPRING_dev-master/data/"
project_dir = path1+'/mamito/'
plot_name = 'all_above_900_UMAP_no_cc_2000'

In [59]:
help(srz.append_color_tracks)

Help on function append_color_tracks in module rz_utility_spring:

append_color_tracks(ctracks, fname, backup=False)
    ctracks - dictionary of continuous colortracks to append
    fname - path to spring directory containing color_stats.json and color_data_gene_sets.csv
    if backup=True, will safe backups of original color_data_gene_sets.csv and color_stats.json files.



### Append continuous colortracks

In [60]:
help(srz.append_color_tracks)

Help on function append_color_tracks in module rz_utility_spring:

append_color_tracks(ctracks, fname, backup=False)
    ctracks - dictionary of continuous colortracks to append
    fname - path to spring directory containing color_stats.json and color_data_gene_sets.csv
    if backup=True, will safe backups of original color_data_gene_sets.csv and color_stats.json files.



In [61]:
conttoappend = newcolobs.iloc[:,:2].to_dict(orient='list')

In [62]:
srz.append_color_tracks(conttoappend,project_dir+plot_name,True)

### Append categorical colotrack

In [63]:
# load current color dictionary
cg0 = srz.read_cell_groupings(project_dir+plot_name+'/categorical_coloring_data.json')

# load cell index
cellix = np.loadtxt(project_dir+plot_name+'/cell_filter.txt',dtype=int)

# color dictionary of dictionaries
cdd = {key:value['label_colors'] for key,value in cg0.items()}

In [64]:
# prepare a new cell grouping dictionary to append:
cols_to_add = ['phase']
print(cols_to_add)

cg = newcolobs.iloc[cellix][cols_to_add].astype(str).to_dict(orient='list')
cg

['phase']


{'phase': ['G0',
  'G2M',
  'G2M',
  'G1S',
  'G1S',
  'G0',
  'G0',
  'G1S',
  'G0',
  'G0',
  'G2M',
  'G0',
  'G1S',
  'G2M',
  'G0',
  'G0',
  'G1S',
  'G2M',
  'G0',
  'G2M',
  'G0',
  'G2M',
  'G1S',
  'G0',
  'G2M',
  'G0',
  'G2M',
  'G1S',
  'G1S',
  'G2M',
  'G0',
  'G0',
  'G2M',
  'G1S',
  'G2M',
  'G2M',
  'G0',
  'G0',
  'G0',
  'G0',
  'G1S',
  'G1S',
  'G2M',
  'G2M',
  'G0',
  'G2M',
  'G2M',
  'G0',
  'G1S',
  'G1S',
  'G2M',
  'G1S',
  'G1S',
  'G1S',
  'G2M',
  'G0',
  'G0',
  'G1S',
  'G0',
  'G0',
  'G2M',
  'G2M',
  'G2M',
  'G1S',
  'G2M',
  'G0',
  'G1S',
  'G1S',
  'G0',
  'G0',
  'G0',
  'G2M',
  'G2M',
  'G2M',
  'G1S',
  'G2M',
  'G0',
  'G2M',
  'G0',
  'G1S',
  'G1S',
  'G2M',
  'G0',
  'G0',
  'G1S',
  'G0',
  'G0',
  'G1S',
  'G0',
  'G0',
  'G0',
  'G1S',
  'G1S',
  'G2M',
  'G0',
  'G0',
  'G0',
  'G2M',
  'G1S',
  'G1S',
  'G2M',
  'G2M',
  'G0',
  'G0',
  'G2M',
  'G0',
  'G2M',
  'G1S',
  'G0',
  'G2M',
  'G2M',
  'G1S',
  'G1S',
  'G2M',
  'G0',
 

In [65]:
# append!
srz.append_cell_groupings(project_dir+plot_name,cg)

In [24]:
!open "$project_dir$plot_name"